# Adding Back the Tides
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

In [2]:
# start_day, end_day = 1, 5
# days = range(start_day, end_day+1)

start = dt.datetime(2019,2,28)
split = dt.datetime(2019,6,1)

In [5]:
# dates for each run
numdays = 2 #16 for the first and last 2 weeks and 15 for the rest
date_list = [start + dt.timedelta(days=x) for x in range(numdays)]

In [6]:
date_list[-1] #this should be the first day for your next run

datetime.datetime(2019, 3, 1, 0, 0)

In [7]:
if start >= split: #since you will need just 1 meesely day before june 1 see if you can just copy May 30 to results2 
    path = Path("/results2/SalishSea/nowcast-green.201812/")
else:
    path = Path("/results/SalishSea/nowcast-green.201812/")

drop_vars = (
    "bounds_lon", "bounds_lat", "area", "deptht_bounds", "PAR",
    "time_centered", "time_centered_bounds", "time_counter_bounds", "dissolved_oxygen",
    "sigma_theta", "Fraser_tracer", "dissolved_inorganic_carbon", "total_alkalnity",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1h_*_carp_T.nc")) for day in date_list]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
e3t = mydata['e3t']

In [8]:
# convert e3t to e3u, e3v, and to e3w
e3t_xshift = e3t.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5
e3u = e3u.rename({'deptht': 'depthu'})

e3t_yshift = e3t.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'deptht': 'depthv'})

In [9]:
drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthu_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1h_*_grid_U.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1h_*_grid_U.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u = mydata['vozocrtx']

In [10]:
drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthv_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1h_*_grid_V.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1h_*_grid_V.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v = mydata['vomecrty']

In [29]:
u[12,1,270,150].values

array(0.0365625)

In [75]:
u[12,1,270,150]

<xarray.DataArray 'vozocrtx' ()>
dask.array<getitem, shape=(), dtype=float32, chunksize=(), chunktype=numpy.ndarray>
Coordinates:
    depthu        float32 1.5
    time_counter  datetime64[ns] 2019-02-28T12:30:00
Attributes:
    standard_name:       sea_water_x_velocity
    long_name:           ocean current along x-axis
    units:               m/s
    online_operation:    average
    interval_operation:  40 s
    interval_write:      1 h
    cell_methods:        time: mean (interval: 40 s)
    cell_measures:       area: area

In [31]:
#calcuate bartropic component of u
ut_h = (u*e3u).sum(dim='depthu')/e3u.sum(dim='depthu')

In [33]:
ut_h[12,270,150].values

/home/rbeutel/anaconda3/envs/OceanParcels/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


array(-0.01994948, dtype=float32)

In [36]:
e3u[12,1,270,150].values

array(1.0029964, dtype=float32)

In [43]:
e3t[12,1,270,150].values

array(1.00301015)

In [37]:
e3u.sum(dim='depthu')[12,270,150].values

array(422.38257, dtype=float32)

In [38]:
(u*e3u).sum(dim='depthu')[12,270,150].values

array(-8.426313, dtype=float32)

In [39]:
(u).sum(dim='depthu')[12,270,150].values

array(-1.3162824, dtype=float32)

In [62]:
#calcuate bartropic component of v
vt_h = (v*e3v).sum(dim='depthv')/e3v.sum(dim='depthv')

u, v, and e3* get reused bellow, so to be safe force the loading of ut_h and vt_h

In [41]:
# Now get the required data from the daily files
drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "deptht_bounds", "PAR",
    "time_centered", "time_centered_bounds", "time_counter_bounds", "dissolved_oxygen",
    "sigma_theta", "Fraser_tracer", "dissolved_inorganic_carbon", "total_alkalnity",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1d_*_carp_T.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_carp_T.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
e3t_d = mydata['e3t']

drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthu_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1d_*_grid_U.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_grid_U.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['vozocrtx']

drop_vars = (
    "nav_lon", "bounds_lon", "nav_lat", "bounds_lat", "area", "depthv_bounds", 
    "time_centered", "time_centered_bounds", "time_counter_bounds",
)

files = [sorted(path.glob("{:%d%b%y}".format(day).lower()+"/SalishSea_1d_*_grid_V.nc")) for day in date_list]
# files = [sorted(path.glob("{:02}mar19/SalishSea_1d_*_grid_V.nc".format(day))) for day in days]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vomecrty']

In [42]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t_d.shift(x=-1,fill_value=0)
e3u_d = e3t_xshift+e3t_d
e3u_d = e3u_d*0.5
e3u_d = e3u_d.rename({'deptht': 'depthu'})

e3t_yshift = e3t_d.shift(y=-1,fill_value=0)
e3v_d = e3t_yshift+e3t_d
e3v_d = e3v_d*0.5
e3v_d = e3v_d.rename({'deptht': 'depthv'})

In [47]:
#calcuate bartropic component
ut_d = (u_d*e3u_d).sum(dim='depthu')/e3u_d.sum(dim='depthu')

In [85]:
print('example values:')
print(f'hourly u strait from model: {u[12,1,270,150].values:.2f}')
print(f'hourly calculated barotropic u: {ut_h[12,270,150].values:.2f}')
print(f'daily u strait from model: {u_d[0,1,270,150].values:.2f}')
print(f'daily calculated barotropic u: {ut_d[0,270,150].values:.2f}')
print(f'daily calculated baroclinic u: {uc_d[0,1,270,150].values:.2f}')
print(f'daily calculated baroclinic u post interpolating (check): {uc_h_interp[0,1,270,150].values:.2f}')
print(f'final calculated u: {u_new[0,270,150,1].values:.2f}')

example values:
hourly u strait from model: 0.04


/home/rbeutel/anaconda3/envs/OceanParcels/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


hourly calculated barotropic u: -0.02
daily u strait from model: -0.22


/home/rbeutel/anaconda3/envs/OceanParcels/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


daily calculated barotropic u: 0.02


...

daily calculated baroclinic u: -0.24


/home/rbeutel/anaconda3/envs/OceanParcels/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


daily calculated baroclinic u post interpolating (check): -0.24


/home/rbeutel/anaconda3/envs/OceanParcels/lib/python3.8/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


final calculated u: -0.26


In [62]:
#subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?

interpolate + resample uc_d to get it in an hourly format

In [63]:
uc_h_interp = uc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [64]:
u_new = ut_h  + uc_h_interp

In [66]:
u_new

<xarray.DataArray (time_counter: 25, y: 898, x: 398, depthu: 40)>
dask.array<add, shape=(25, 898, 398, 40), dtype=float32, chunksize=(13, 898, 398, 40), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2019-02-28T12:30:00 ... 2019-...
    nav_lat       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
  * depthu        (depthu) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
Dimensions without coordinates: y, x

In [ ]:
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))

In [ ]:
#calcuate bartropic component
vt_d = (v*e3v).sum(dim='depthv')/e3v.sum(dim='depthv')

In [ ]:
#subtract from v to get baroclinic component
vc_d = v-vt_d 

In [ ]:
vc_d.load(scheduler="processes", num_workers=n)

In [ ]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [ ]:
v_new = vt_h  + vc_h_interp

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))